In [7]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from ipywidgets import interactive, interactive_output, fixed, IntSlider, FloatSlider
import ipywidgets as widgets
import unittest

In [8]:
class Integral:
    
    def quad(f, a, b, N=10000):
    
        x = np.linspace(a, b, N)
        s = sum([f(i) for i in x])
        return s*abs(b-a)/N

In [23]:
class Laguerre:
    def __init__(self, beta=2.0, sigma=4.0):
        self._beta = beta
        self._sigma = sigma
    
    @property
    def sigma(self):
        return self._sigma
    
    @sigma.setter
    def sigma(self, value):
        if not isinstance(value, (int, float)):
            raise ValueError("sigma must be a number")
        elif self.beta > value:
            raise ValueError("sigma must be greater than beta")
        self._sigma = value
    
    @property
    def beta(self):
        return self._beta
    
    @beta.setter
    def beta(self, value):
        if not isinstance(value, (int, float)):
            raise ValueError("beta must be a number")
        elif self.sigma < value:
            raise ValueError("beta must be less than sigma")
        self._beta = value
    
    def laguerre(self, t, n):
    
        #Перевірка вхідних даних
        if n < 0:
            raise ValueError('Value "n" must be positive')

        if self.beta < 0:
            raise ValueError('Value "beta" must be positive')

        if self.sigma < self.beta:
            raise ValueError('Value "sigma" must be greater than beta')


        #Обчислення значення функції Лаґерра при n = 0 та n = 1
        l_0 = np.sqrt(self.sigma) * (np.exp(-self.beta * t / 2))
        l_1 = np.sqrt(self.sigma) * (1 - self.sigma * t) * (np.exp(-self.beta  * t / 2))

        if n == 0:
            return l_0
        if n == 1:
            return l_1
        #Обчислення значення функції Лаґерра при n>=2
        if n >= 2:
            l_next = (2 * 2 - 1 - t * self.sigma) / 2 * l_1 - (2 - 1) / 2 * l_0
            for j in range(3, n+1):
                l_0 = l_1
                l_1 = l_next
                l_next = (2 * j - 1 - t * self.sigma) / j * l_1 - (j - 1) / j * l_0
            return l_next
    
    def tabulate_laguerre(self, T, t_step = 0.01, n=2):
    
        #Перевірка вхідних даних
        if t_step < 0:
            raise ValueError('Value "T step" must be positive')
        if n < 0:
            raise ValueError('Value "n" must be positive')

        if self.beta < 0:
            raise ValueError('Value "beta" must be positive')

        if self.sigma < self.beta:
            raise ValueError('Value "sigma" must be greater than beta')

        t = np.arange(0, T, t_step)
        return t, self.laguerre( t, n)
    
    def laguerre_experiment(self, T, epsilon=0.001, N=20):

        #Перевірка вхідних даних
        if self.beta < 0:
            raise ValueError('Value "beta" must be positive')

        if self.sigma < self.beta:
            raise ValueError('Value "sigma" must be greater than beta')

        if epsilon < 0:
            raise ValueError('Value "epsilon" must be positive')

        if N < 0:
            raise ValueError('Value "N" must be positive')

        t = np.linspace(0, T, 1000)
        n = range(0, N+1)
        result = None
        for i in t:
            flag = True
            for j in n:
                if abs(self.laguerre( i, j)) > epsilon:
                    flag = False
                    break
            if flag and result is None:
                result = i


        cols = {"t" : t}
        for j in n:
            cols[f"n={j}"] = self.laguerre(t, j)

        df = pd.DataFrame(cols)

        return result, df.round(5)
    def laguerre_transform(self, f, n):
    
        if n < 0:
            raise ValueError('Value "n" must be positive')

        # Функція для інтегрування
        def integrand(t):
            return f(t)*self.laguerre( t, n)*np.exp(-t*(self.sigma-self.beta))

        # Верхня межа інтегрування
        b = self.laguerre_experiment(100)[0]

        return Integral.quad(integrand, 0, b)
    
    def tabulate_tranformation(self,f, N):
        t = range(0, N+1)
        results = [self.laguerre_transform(f, n) for n in t]
        df = pd.DataFrame({"n": t, "result": results})
        return df
    
    def inverse_laguerre_transform(self,h, t):
        # Обчислення
        return sum([h[k] * self.laguerre(t, k,) for k in range(0, len(h))])
    
    def laguerre_graph(self,t, n, t_step=0.1):
        plt.figure(figsize=(12, 8))

        for n in range(0, n + 1):
            t_data, data = self.tabulate_laguerre(t, t_step, n)
            plt.plot(t_data, data, label="$L_{" + str(n) + "}$")

        plt.title('Многочлени Лагера')
        plt.grid()
        plt.legend(loc="upper left", bbox_to_anchor=(-0.2, 1))
        plt.xlabel("t")
        plt.ylabel("$L_n(t)$")

        plt.savefig("L_n_plot")
        plt.show()
        
    def laguerre_transform_graph(self, f, n, name, t = np.pi * 2, t_step = 0.01):
        # Обчислення послідовності h
        h = self.tabulate_tranformation(f, n)["result"].tolist()

        # Табулювання
        T = np.arange(0, t, t_step)
        inverse_laguerre_transform_tabulation = pd.DataFrame({"t": T,"h": [self.inverse_laguerre_transform(h, t) for t in T]})

        # Побудова графіків
        plt.figure(figsize=(12, 8))

        plt.plot(inverse_laguerre_transform_tabulation["t"],inverse_laguerre_transform_tabulation["h"],)

        plt.title('Обернене перетворення Лагера')
        plt.xlabel('t')
        plt.ylabel(r'$\widetilde{f}^N(t)$')
        plt.grid()
        plt.savefig(name)
        plt.show()

In [44]:
class TestLaguarre(unittest.TestCase):
    def setUp(self):
        self.lag1 = Laguerre()
    def tearDown(self):
        self.lag1 = None
    
    def test_getter(self):
        self.assertEqual(self.lag1.beta, 2)
        self.assertEqual(self.lag1.sigma, 4)
        
    def test_setter_value_error(self):
        with self.assertRaises(ValueError):
            self.lag1.beta = "a"
        
        with self.assertRaises(ValueError):
            self.lag1.sigma = "b"
        
        with self.assertRaises(ValueError):
            self.lag1.beta = 8
        
        with self.assertRaises(ValueError):
            self.lag1.sigma = 0
            
    def test_setter(self):
        self.lag1.sigma = 10
        self.assertEqual(self.lag1.sigma,10)
        
        self.lag1.beta = 7
        self.assertEqual(self.lag1.beta, 7)
        
    def test_laguerre(self):
        result = self.lag1.laguerre(7, 5)
        self.assertEqual(result, -87.85447767364639)
        
    def test_tabulate_laguerre(self):
        res1, res2 = self.lag1.tabulate_laguerre(7)
        self.assertEqual(res1[0], 0)
        self.assertEqual(res2[0],2)
    
    def test_laguerre_experiment(self):
        res1, res2 = self.lag1.laguerre_experiment(100)
        self.assertEqual(res1, 79.07907907907908)
        self.assertEqual(res2['t'][0], 0)
        
    def test_laguerre_transform(self):
        f = lambda t: np.exp(-t**2/2)
        result = self.lag1.laguerre_transform(f, 6)
        self.assertEqual(result, 0.00790221830010673)
    
    def test_inverse_laguerre_transform(self):
        def f(t):
            if t >= 0 and t <= 2*np.pi:
                return np.sin(t-np.pi/2) + 1
            else:
                return 0
        h = self.lag1.tabulate_tranformation(f, 20)["result"].tolist()
        result = self.lag1.inverse_laguerre_transform(h, 2)
        self.assertEqual(result, 1.4159846887860223)

In [45]:
unittest.main(argv=[''], exit=False)

........
----------------------------------------------------------------------
Ran 8 tests in 5.844s

OK
